```
QUERYNAME = CholesterolMSMS;
DEFINE PR = 'C[41..49] H[30..200] O[2] N[1]' WITH DBR = (4.5,10.5), CHG = +1;
DEFINE FR ='C[27] H[45]' WITH DBR = (1,8), CHG = +1;

IDENTIFY

	PR IN MS1+ AND
	FR in MS2+

SUCHTHAT
	(avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
	PR.chemsc[C]/PR.chemsc[db] >= 4.6

REPORT 

	SPECIE = "CE %d:%d" % "((PR.chemsc)[C] - 27, (PR.chemsc)[db] - 4.5)";  
	CLASS = "CE";
	MASS = PR.mass;
	ISOBARIC = PR.isobaric;
	CHEMSC = PR.chemsc;
	ERRppm = "%2.2f" % "(PR.errppm)";
	FRERRppm = "%2.2f" % "(FR.errppm)";
	INT = FR.intensity;
	QUALA = PR.intensity;
```

In [1]:
from lx2.targets import MFQL_util
from lx2.MS_reader import SpectraUtil

In [2]:
QUERYNAME = 'CholesterolMSMS'

In [3]:
elements = {'C':(41,49),'H':(30,200),'O':(2,2),'N':(1,1),'P':(1,1) }      
PR = MFQL_util(elements)
PR.set_dbr(4.5,10.5)

In [4]:
elements = {'C':(27,27),'H':(45,45) }      
FR = MFQL_util(elements)
FR.set_dbr(1,8)

In [15]:
filename = 'test_resources\\small_test\\190321_Serum_Lipidextract_368723_01.mzML'
spectraUtil = SpectraUtil.fromFile(filename, test_sample=True)

In [16]:
MS1 = spectraUtil
MS2 = spectraUtil.get_reset_copy()
MS1.set_mode()
MS1.set_ms_level()
MS2.set_mode()
MS2.set_ms_level(2)


a copy of the original with nothing set... sorry no undo
set mode to positive : True, (false means negative) 
set ms level to  : 1
set mode to positive : True, (false means negative) 
set ms level to  : 2


In [20]:
MS1.get_nearest(PR._df)

find the nearest Peaks to the target_peaks with a tolerance of 0.01


,id,m,i,index,C,H,O,N,P,dbr,chem
24825,controllerType=0 controllerNumber=1 scan=8,633.459106,55058.835938,34.0,41.0,64.0,2.0,1.0,1.0,10.5,C41 H64 O2 N1 P1
24826,controllerType=0 controllerNumber=1 scan=7,633.464111,3056.083740,34.0,41.0,64.0,2.0,1.0,1.0,10.5,C41 H64 O2 N1 P1
24827,controllerType=0 controllerNumber=1 scan=8,633.472656,93704.203125,34.0,41.0,64.0,2.0,1.0,1.0,10.5,C41 H64 O2 N1 P1
24828,controllerType=0 controllerNumber=1 scan=6,633.476074,237.615753,34.0,41.0,64.0,2.0,1.0,1.0,10.5,C41 H64 O2 N1 P1
24955,controllerType=0 controllerNumber=1 scan=4,635.475891,172.791199,36.0,41.0,66.0,2.0,1.0,1.0,9.5,C41 H66 O2 N1 P1
...,...,...,...,...,...,...,...,...,...,...,...
32071,controllerType=0 controllerNumber=1 scan=30,750.640747,50162.398438,1423.0,49.0,85.0,2.0,1.0,1.0,8.0,C49 H85 O2 N1 P1
32072,controllerType=0 controllerNumber=1 scan=15,750.641357,127782.960938,1423.0,49.0,85.0,2.0,1.0,1.0,8.0,C49 H85 O2 N1 P1
32336,controllerType=0 controllerNumber=1 scan=30,753.664856,55418.605469,1426.0,49.0,88.0,2.0,1.0,1.0,6.5,C49 H88 O2 N1 P1
32426,controllerType=0 controllerNumber=1 scan=26,754.667053,135862.843750,1427.0,49.0,89.0,2.0,1.0,1.0,6.0,C49 H89 O2 N1 P1


In [21]:
MS2.get_nearest(FR._df)

find the nearest Peaks to the target_peaks with a tolerance of 0.01


,id,m,i,index,C,H,dbr,chem
7053,controllerType=0 controllerNumber=1 scan=7,369.347656,9.832830e+02,0.0,27.0,45.0,5.5,C27 H45
7054,controllerType=0 controllerNumber=1 scan=11,369.349915,1.620340e+06,0.0,27.0,45.0,5.5,C27 H45
7055,controllerType=0 controllerNumber=1 scan=7,369.350708,1.701846e+03,0.0,27.0,45.0,5.5,C27 H45
7056,controllerType=0 controllerNumber=1 scan=8,369.351044,4.216351e+05,0.0,27.0,45.0,5.5,C27 H45
7057,controllerType=0 controllerNumber=1 scan=50,369.351440,3.547369e+06,0.0,27.0,45.0,5.5,C27 H45
7058,controllerType=0 controllerNumber=1 scan=85,369.351501,2.246635e+03,0.0,27.0,45.0,5.5,C27 H45
7059,controllerType=0 controllerNumber=1 scan=27,369.352081,1.888654e+07,0.0,27.0,45.0,5.5,C27 H45
7060,controllerType=0 controllerNumber=1 scan=16,369.352112,2.761480e+07,0.0,27.0,45.0,5.5,C27 H45
7061,controllerType=0 controllerNumber=1 scan=12,369.352112,2.262215e+07,0.0,27.0,45.0,5.5,C27 H45
7062,controllerType=0 controllerNumber=1 scan=28,369.352112,2.082530e+07,0.0,27.0,45.0,5.5,C27 H45
